classification of Astroid as Hazardous or not using: Neural Network and compare it with KNN method.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from zipfile import ZipFile
from pathlib import Path

In [ ]:
with ZipFile("/content/drive/MyDrive/Colab Notebooks/datasets/astroid.zip","r") as f:
  f.extractall("/content/drive/MyDrive/Colab Notebooks/datasets/astroid")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 1.Data Cleaning

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/astroid/nasa.csv")
df.head(5)

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,True
1,3723955,3723955,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,...,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,False
2,2446862,2446862,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,...,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,True
3,3092506,3092506,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,...,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,False
4,3514799,3514799,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,True


In [ ]:
df.shape

(4687, 40)

In [ ]:
if list(df["Name"] == df["Neo Reference ID"]).count(True) == df.shape[0]:
  print(" Same")

 Same


In [ ]:
# Dropping Neo Ref ID column as it is same as Name
df = df.drop(["Neo Reference ID","Name"],axis=1)

In [ ]:
df.head(5)

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),Est Dia in Feet(max),Close Approach Date,...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,933.308089,1995-01-01,...,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,True
1,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063,1995-01-01,...,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,False
2,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,1698.341531,1995-01-08,...,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,True
3,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,64.569144,1995-01-15,...,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,False
4,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,933.308089,1995-01-15,...,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,True


In [ ]:
df["Equinox"].value_counts()

Equinox
J2000    4687
Name: count, dtype: int64

In [ ]:
# Dropping Equinox column as it has same value for all rows
df = df.drop(["Equinox"],axis=1)

In [ ]:
Y = df["Hazardous"].copy()

In [ ]:
df = df.drop(["Hazardous"],axis=1)

In [ ]:
df.head(5)

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),Est Dia in Feet(max),Close Approach Date,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
0,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,933.308089,1995-01-01,...,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551
1,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063,1995-01-01,...,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330
2,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,1698.341531,1995-01-08,...,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371
3,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,64.569144,1995-01-15,...,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277
4,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,933.308089,1995-01-15,...,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Absolute Magnitude            4687 non-null   float64
 1   Est Dia in KM(min)            4687 non-null   float64
 2   Est Dia in KM(max)            4687 non-null   float64
 3   Est Dia in M(min)             4687 non-null   float64
 4   Est Dia in M(max)             4687 non-null   float64
 5   Est Dia in Miles(min)         4687 non-null   float64
 6   Est Dia in Miles(max)         4687 non-null   float64
 7   Est Dia in Feet(min)          4687 non-null   float64
 8   Est Dia in Feet(max)          4687 non-null   float64
 9   Close Approach Date           4687 non-null   object 
 10  Epoch Date Close Approach     4687 non-null   int64  
 11  Relative Velocity km per sec  4687 non-null   float64
 12  Relative Velocity km per hr   4687 non-null   float64
 13  Mil

In [ ]:
df["Close Approach Date"].head(5)

0    1995-01-01
1    1995-01-01
2    1995-01-08
3    1995-01-15
4    1995-01-15
Name: Close Approach Date, dtype: object

In [ ]:
df["Close Approach Date"].value_counts()

Close Approach Date
2016-07-22    18
2015-01-15    17
2015-02-15    16
2012-03-15    16
2014-11-22    16
              ..
2000-05-08     1
1999-05-15     1
1998-09-15     1
1999-04-22     1
2001-06-22     1
Name: count, Length: 777, dtype: int64

In [ ]:
df["Orbiting Body"].head(5)

0    Earth
1    Earth
2    Earth
3    Earth
4    Earth
Name: Orbiting Body, dtype: object

In [ ]:
df["Orbiting Body"].value_counts()

Orbiting Body
Earth    4687
Name: count, dtype: int64

In [ ]:
df = df.drop(["Orbiting Body"],axis=1)

In [ ]:
df = df.drop(["Close Approach Date"],axis=1)

In [ ]:
df.shape

(4687, 34)

In [ ]:
df['Orbit Determination Date'] = pd.to_datetime(df['Orbit Determination Date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Absolute Magnitude            4687 non-null   float64       
 1   Est Dia in KM(min)            4687 non-null   float64       
 2   Est Dia in KM(max)            4687 non-null   float64       
 3   Est Dia in M(min)             4687 non-null   float64       
 4   Est Dia in M(max)             4687 non-null   float64       
 5   Est Dia in Miles(min)         4687 non-null   float64       
 6   Est Dia in Miles(max)         4687 non-null   float64       
 7   Est Dia in Feet(min)          4687 non-null   float64       
 8   Est Dia in Feet(max)          4687 non-null   float64       
 9   Epoch Date Close Approach     4687 non-null   int64         
 10  Relative Velocity km per sec  4687 non-null   float64       
 11  Relative Velocity km per hr   

In [ ]:
for col in df.columns:
  print(f"--------------Column: {col} ------------------------")
  print(df[col].value_counts())

In [ ]:
df.corr()


,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),Est Dia in Feet(max),Epoch Date Close Approach,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
Absolute Magnitude,1.000000,-0.613482,-0.613482,-0.613482,-0.613482,-0.613482,-0.613482,-0.613482,-0.613482,0.168621,...,-0.212437,-0.459632,-0.011470,-0.206774,0.086966,0.031784,-0.256169,-0.115855,-0.049401,0.195652
Est Dia in KM(min),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in KM(max),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in M(min),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in M(max),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in Miles(min),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in Miles(max),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in Feet(min),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in Feet(max),-0.613482,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.094121,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Epoch Date Close Approach,0.168621,-0.094121,-0.094121,-0.094121,-0.094121,-0.094121,-0.094121,-0.094121,-0.094121,1.000000,...,0.134480,-0.067337,-0.019341,0.130175,0.131854,0.001834,0.114616,-0.015533,-0.026182,-0.137663


In [ ]:
df[["Est Dia in KM(min)","Est Dia in KM(max)",	"Est Dia in M(min)","Est Dia in M(max)",	"Est Dia in Miles(min)","Est Dia in Miles(max)","Est Dia in Feet(min)"]].head()

,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min)
0,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066
1,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620
2,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423
3,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199
4,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066


In [ ]:
df = df.drop(["Est Dia in M(min)","Est Dia in M(max)",	"Est Dia in Miles(min)","Est Dia in Miles(max)","Est Dia in Feet(min)"],axis=1)
df.head()

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in Feet(max),Epoch Date Close Approach,Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour,Miss Dist.(Astronomical),Miss Dist.(lunar),...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
0,21.6,0.127220,0.284472,933.308089,788947200000,6.115834,22017.003799,13680.509944,0.419483,163.178711,...,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551
1,21.3,0.146068,0.326618,1071.581063,788947200000,18.113985,65210.346095,40519.173105,0.383014,148.992630,...,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330
2,20.3,0.231502,0.517654,1698.341531,789552000000,7.590711,27326.560182,16979.661798,0.050956,19.821890,...,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371
3,27.4,0.008801,0.019681,64.569144,790156800000,11.173874,40225.948191,24994.839864,0.285322,110.990387,...,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277
4,21.6,0.127220,0.284472,933.308089,790156800000,9.840831,35426.991794,22012.954985,0.407832,158.646713,...,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395


In [ ]:
df["Epoch Date Close Approach"].value_counts()

Epoch Date Close Approach
1469170800000    18
1421308800000    17
1423987200000    16
1331794800000    16
1416643200000    16
                 ..
957769200000      1
926751600000      1
905842800000      1
924764400000      1
993193200000      1
Name: count, Length: 777, dtype: int64

In [ ]:
df.corr()

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in Feet(max),Epoch Date Close Approach,Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour,Miss Dist.(Astronomical),Miss Dist.(lunar),...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
Absolute Magnitude,1.000000,-0.613482,-0.613482,-0.613482,0.168621,-0.376853,-0.376853,-0.376853,-0.339117,-0.339117,...,-0.212437,-0.459632,-0.011470,-0.206774,0.086966,0.031784,-0.256169,-0.115855,-0.049401,0.195652
Est Dia in KM(min),-0.613482,1.000000,1.000000,1.000000,-0.094121,0.242141,0.242141,0.242141,0.188027,0.188027,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in KM(max),-0.613482,1.000000,1.000000,1.000000,-0.094121,0.242141,0.242141,0.242141,0.188027,0.188027,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in Feet(max),-0.613482,1.000000,1.000000,1.000000,-0.094121,0.242141,0.242141,0.242141,0.188027,0.188027,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Epoch Date Close Approach,0.168621,-0.094121,-0.094121,-0.094121,1.000000,-0.105725,-0.105725,-0.105725,-0.144807,-0.144807,...,0.134480,-0.067337,-0.019341,0.130175,0.131854,0.001834,0.114616,-0.015533,-0.026182,-0.137663
Relative Velocity km per sec,-0.376853,0.242141,0.242141,0.242141,-0.105725,1.000000,1.000000,1.000000,0.351694,0.351694,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Relative Velocity km per hr,-0.376853,0.242141,0.242141,0.242141,-0.105725,1.000000,1.000000,1.000000,0.351694,0.351694,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Miles per hour,-0.376853,0.242141,0.242141,0.242141,-0.105725,1.000000,1.000000,1.000000,0.351694,0.351694,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Miss Dist.(Astronomical),-0.339117,0.188027,0.188027,0.188027,-0.144807,0.351694,0.351694,0.351694,1.000000,1.000000,...,-0.112158,0.255828,-0.023033,-0.109888,-0.079937,-0.021483,-0.103231,0.189409,-0.010157,0.104642
Miss Dist.(lunar),-0.339117,0.188027,0.188027,0.188027,-0.144807,0.351694,0.351694,0.351694,1.000000,1.000000,...,-0.112158,0.255828,-0.023033,-0.109888,-0.079937,-0.021483,-0.103231,0.189409,-0.010157,0.104642


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Absolute Magnitude            4687 non-null   float64       
 1   Est Dia in KM(min)            4687 non-null   float64       
 2   Est Dia in KM(max)            4687 non-null   float64       
 3   Est Dia in Feet(max)          4687 non-null   float64       
 4   Epoch Date Close Approach     4687 non-null   int64         
 5   Relative Velocity km per sec  4687 non-null   float64       
 6   Relative Velocity km per hr   4687 non-null   float64       
 7   Miles per hour                4687 non-null   float64       
 8   Miss Dist.(Astronomical)      4687 non-null   float64       
 9   Miss Dist.(lunar)             4687 non-null   float64       
 10  Miss Dist.(kilometers)        4687 non-null   float64       
 11  Miss Dist.(miles)             

In [ ]:
df[["Miss Dist.(Astronomical)","Miss Dist.(lunar)","Miss Dist.(kilometers)","Miss Dist.(miles)"]]

,Miss Dist.(Astronomical),Miss Dist.(lunar),Miss Dist.(kilometers),Miss Dist.(miles)
0,0.419483,163.178711,6.275369e+07,3.899334e+07
1,0.383014,148.992630,5.729815e+07,3.560342e+07
2,0.050956,19.821890,7.622912e+06,4.736658e+06
3,0.285322,110.990387,4.268362e+07,2.652237e+07
4,0.407832,158.646713,6.101082e+07,3.791037e+07
...,...,...,...,...
4682,0.041361,16.089413,6.187511e+06,3.844741e+06
4683,0.006469,2.516399,9.677324e+05,6.013211e+05
4684,0.061009,23.732393,9.126775e+06,5.671115e+06
4685,0.260760,101.435493,3.900908e+07,2.423912e+07


In [ ]:
df = df.drop(["Miss Dist.(lunar)","Miss Dist.(kilometers)","Miss Dist.(miles)"],axis=1)
df.corr()

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in Feet(max),Epoch Date Close Approach,Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour,Miss Dist.(Astronomical),Orbit ID,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
Absolute Magnitude,1.000000,-0.613482,-0.613482,-0.613482,0.168621,-0.376853,-0.376853,-0.376853,-0.339117,-0.575668,...,-0.212437,-0.459632,-0.011470,-0.206774,0.086966,0.031784,-0.256169,-0.115855,-0.049401,0.195652
Est Dia in KM(min),-0.613482,1.000000,1.000000,1.000000,-0.094121,0.242141,0.242141,0.242141,0.188027,0.724089,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in KM(max),-0.613482,1.000000,1.000000,1.000000,-0.094121,0.242141,0.242141,0.242141,0.188027,0.724089,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in Feet(max),-0.613482,1.000000,1.000000,1.000000,-0.094121,0.242141,0.242141,0.242141,0.188027,0.724089,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Epoch Date Close Approach,0.168621,-0.094121,-0.094121,-0.094121,1.000000,-0.105725,-0.105725,-0.105725,-0.144807,-0.084657,...,0.134480,-0.067337,-0.019341,0.130175,0.131854,0.001834,0.114616,-0.015533,-0.026182,-0.137663
Relative Velocity km per sec,-0.376853,0.242141,0.242141,0.242141,-0.105725,1.000000,1.000000,1.000000,0.351694,0.110679,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Relative Velocity km per hr,-0.376853,0.242141,0.242141,0.242141,-0.105725,1.000000,1.000000,1.000000,0.351694,0.110679,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Miles per hour,-0.376853,0.242141,0.242141,0.242141,-0.105725,1.000000,1.000000,1.000000,0.351694,0.110679,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Miss Dist.(Astronomical),-0.339117,0.188027,0.188027,0.188027,-0.144807,0.351694,0.351694,0.351694,1.000000,0.149814,...,-0.112158,0.255828,-0.023033,-0.109888,-0.079937,-0.021483,-0.103231,0.189409,-0.010157,0.104642
Orbit ID,-0.575668,0.724089,0.724089,0.724089,-0.084657,0.110679,0.110679,0.110679,0.149814,1.000000,...,0.005462,0.112125,0.047959,0.002705,-0.063884,0.001338,0.022269,0.037901,0.048985,-0.008898


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Absolute Magnitude            4687 non-null   float64       
 1   Est Dia in KM(min)            4687 non-null   float64       
 2   Est Dia in KM(max)            4687 non-null   float64       
 3   Est Dia in Feet(max)          4687 non-null   float64       
 4   Epoch Date Close Approach     4687 non-null   int64         
 5   Relative Velocity km per sec  4687 non-null   float64       
 6   Relative Velocity km per hr   4687 non-null   float64       
 7   Miles per hour                4687 non-null   float64       
 8   Miss Dist.(Astronomical)      4687 non-null   float64       
 9   Orbit ID                      4687 non-null   int64         
 10  Orbit Determination Date      4687 non-null   datetime64[ns]
 11  Orbit Uncertainity            

In [ ]:
df = df.drop(["Est Dia in Feet(max)"],axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Absolute Magnitude            4687 non-null   float64       
 1   Est Dia in KM(min)            4687 non-null   float64       
 2   Est Dia in KM(max)            4687 non-null   float64       
 3   Epoch Date Close Approach     4687 non-null   int64         
 4   Relative Velocity km per sec  4687 non-null   float64       
 5   Relative Velocity km per hr   4687 non-null   float64       
 6   Miles per hour                4687 non-null   float64       
 7   Miss Dist.(Astronomical)      4687 non-null   float64       
 8   Orbit ID                      4687 non-null   int64         
 9   Orbit Determination Date      4687 non-null   datetime64[ns]
 10  Orbit Uncertainity            4687 non-null   int64         
 11  Minimum Orbit Intersection    

In [ ]:
df[["Relative Velocity km per sec","Relative Velocity km per hr","Miles per hour"]]

,Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour
0,6.115834,22017.003799,13680.509944
1,18.113985,65210.346095,40519.173105
2,7.590711,27326.560182,16979.661798
3,11.173874,40225.948191,24994.839864
4,9.840831,35426.991794,22012.954985
...,...,...,...
4682,22.154265,79755.354273,49556.875548
4683,3.225150,11610.539577,7214.337772
4684,7.191642,25889.910626,16086.983633
4685,11.352090,40867.522309,25393.489071


In [ ]:
df = df.drop(["Relative Velocity km per hr","Miles per hour"],axis =1)

In [ ]:
df.corr()

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Epoch Date Close Approach,Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit ID,Orbit Determination Date,Orbit Uncertainity,Minimum Orbit Intersection,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
Absolute Magnitude,1.000000,-0.613482,-0.613482,0.168621,-0.376853,-0.339117,-0.575668,-0.057220,0.677764,-0.488235,...,-0.212437,-0.459632,-0.011470,-0.206774,0.086966,0.031784,-0.256169,-0.115855,-0.049401,0.195652
Est Dia in KM(min),-0.613482,1.000000,1.000000,-0.094121,0.242141,0.188027,0.724089,0.082581,-0.399488,0.257904,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Est Dia in KM(max),-0.613482,1.000000,1.000000,-0.094121,0.242141,0.188027,0.724089,0.082581,-0.399488,0.257904,...,0.121224,0.259450,0.036558,0.118314,-0.071866,-0.019577,0.151836,0.062167,0.031455,-0.104350
Epoch Date Close Approach,0.168621,-0.094121,-0.094121,1.000000,-0.105725,-0.144807,-0.084657,-0.043989,0.216463,-0.022532,...,0.134480,-0.067337,-0.019341,0.130175,0.131854,0.001834,0.114616,-0.015533,-0.026182,-0.137663
Relative Velocity km per sec,-0.376853,0.242141,0.242141,-0.105725,1.000000,0.351694,0.110679,0.065817,-0.202234,0.076306,...,0.014351,0.514657,-0.021301,0.017961,-0.506978,-0.002913,0.144782,0.020006,0.017685,0.022452
Miss Dist.(Astronomical),-0.339117,0.188027,0.188027,-0.144807,0.351694,1.000000,0.149814,0.038579,-0.326158,0.351981,...,-0.112158,0.255828,-0.023033,-0.109888,-0.079937,-0.021483,-0.103231,0.189409,-0.010157,0.104642
Orbit ID,-0.575668,0.724089,0.724089,-0.084657,0.110679,0.149814,1.000000,0.054764,-0.504578,0.124283,...,0.005462,0.112125,0.047959,0.002705,-0.063884,0.001338,0.022269,0.037901,0.048985,-0.008898
Orbit Determination Date,-0.057220,0.082581,0.082581,-0.043989,0.065817,0.038579,0.054764,1.000000,-0.066948,0.023369,...,-0.009157,0.035780,0.034721,-0.019720,-0.023291,0.001642,-0.004163,0.117350,-0.020953,-0.016262
Orbit Uncertainity,0.677764,-0.399488,-0.399488,0.216463,-0.202234,-0.326158,-0.504578,-0.066948,1.000000,-0.267524,...,0.043902,-0.228806,-0.009618,0.047535,0.107946,0.016307,0.020908,-0.325784,-0.059396,-0.023776
Minimum Orbit Intersection,-0.488235,0.257904,0.257904,-0.022532,0.076306,0.351981,0.124283,0.023369,-0.267524,1.000000,...,0.288947,0.439867,-0.008963,0.279885,0.299991,-0.029130,0.242023,0.080592,-0.013336,-0.290538


In [ ]:
df["Orbit ID"].value_counts()

Orbit ID
8      235
9      227
7      212
10     205
11     198
      ... 
130      1
170      1
324      1
127      1
107      1
Name: count, Length: 188, dtype: int64

In [ ]:
df["Orbit Uncertainity"].value_counts()

Orbit Uncertainity
0    1353
7     700
6     620
1     568
8     329
5     321
2     279
4     212
3     182
9     123
Name: count, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Absolute Magnitude            4687 non-null   float64       
 1   Est Dia in KM(min)            4687 non-null   float64       
 2   Est Dia in KM(max)            4687 non-null   float64       
 3   Epoch Date Close Approach     4687 non-null   int64         
 4   Relative Velocity km per sec  4687 non-null   float64       
 5   Miss Dist.(Astronomical)      4687 non-null   float64       
 6   Orbit ID                      4687 non-null   int64         
 7   Orbit Determination Date      4687 non-null   datetime64[ns]
 8   Orbit Uncertainity            4687 non-null   int64         
 9   Minimum Orbit Intersection    4687 non-null   float64       
 10  Jupiter Tisserand Invariant   4687 non-null   float64       
 11  Epoch Osculation              

In [ ]:
for col in df.columns:
  print(f"{col} : {df[col].nunique()}")

Absolute Magnitude : 269
Est Dia in KM(min) : 269
Est Dia in KM(max) : 269
Epoch Date Close Approach : 777
Relative Velocity km per sec : 4687
Miss Dist.(Astronomical) : 4673
Orbit ID : 188
Orbit Determination Date : 2680
Orbit Uncertainity : 10
Minimum Orbit Intersection : 3678
Jupiter Tisserand Invariant : 2495
Epoch Osculation : 429
Eccentricity : 3692
Semi Major Axis : 3692
Inclination : 3692
Asc Node Longitude : 3692
Orbital Period : 3692
Perihelion Distance : 3692
Perihelion Arg : 3692
Aphelion Dist : 3692
Perihelion Time : 3692
Mean Anomaly : 3692
Mean Motion : 3692


In [ ]:
df[["Absolute Magnitude","Est Dia in KM(min)","Est Dia in KM(max)"]]

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max)
0,21.600,0.127220,0.284472
1,21.300,0.146068,0.326618
2,20.300,0.231502,0.517654
3,27.400,0.008801,0.019681
4,21.600,0.127220,0.284472
...,...,...,...
4682,23.900,0.044112,0.098637
4683,28.200,0.006089,0.013616
4684,22.700,0.076658,0.171412
4685,21.800,0.116026,0.259442


In [ ]:
df.describe()

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Epoch Date Close Approach,Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit ID,Orbit Determination Date,Orbit Uncertainity,Minimum Orbit Intersection,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
count,4687.000000,4687.000000,4687.000000,4.687000e+03,4687.000000,4687.000000,4687.000000,4687,4687.000000,4687.000000,...,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4.687000e+03,4687.000000,4687.000000
mean,22.267865,0.204604,0.457509,1.179881e+12,13.970811,0.256778,28.300619,2017-04-16 09:54:40.266908672,3.516962,0.082320,...,1.400264,13.373844,172.157275,635.582076,0.813383,183.932151,1.987144,2.457728e+06,181.167927,0.738242
min,11.160000,0.001011,0.002260,7.889472e+11,0.335504,0.000178,1.000000,2014-06-13 15:20:44,0.000000,0.000002,...,0.615920,0.014513,0.001941,176.557161,0.080744,0.006918,0.803765,2.450100e+06,0.003191,0.086285
25%,20.100000,0.033462,0.074824,1.015574e+12,8.432865,0.133420,9.000000,2017-04-06 08:42:07.500000,0.000000,0.014585,...,1.000635,4.962341,83.081208,365.605031,0.630834,95.625916,1.266059,2.457815e+06,87.006918,0.453289
50%,21.900000,0.110804,0.247765,1.203062e+12,12.917889,0.265029,16.000000,2017-04-06 09:03:02,3.000000,0.047365,...,1.240981,10.311836,172.625393,504.947292,0.833153,189.761641,1.618195,2.457973e+06,185.718889,0.712946
75%,24.500000,0.253837,0.567597,1.355558e+12,18.077649,0.384154,31.000000,2017-04-06 09:22:13,6.000000,0.123593,...,1.678364,19.511681,255.026909,794.195972,0.997227,271.777557,2.451171,2.458108e+06,276.531946,0.984669
max,32.100000,15.579552,34.836938,1.473318e+12,44.633747,0.499884,611.000000,2017-09-28 06:49:30,9.000000,0.477891,...,5.072008,75.406667,359.905890,4172.231343,1.299832,359.993098,8.983852,2.458839e+06,359.917991,2.039000
std,2.890972,0.369573,0.826391,1.981540e+11,7.293223,0.145798,38.299668,NaN,3.078307,0.090300,...,0.524154,10.936227,103.276777,370.954727,0.242059,103.513035,0.951519,9.442264e+02,107.501623,0.342627


# 2.Prepare the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df["Orbit Uncertainity"].value_counts()

Orbit Uncertainity
0    1353
7     700
6     620
1     568
8     329
5     321
2     279
4     212
3     182
9     123
Name: count, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Absolute Magnitude            4687 non-null   float64       
 1   Est Dia in KM(min)            4687 non-null   float64       
 2   Est Dia in KM(max)            4687 non-null   float64       
 3   Epoch Date Close Approach     4687 non-null   int64         
 4   Relative Velocity km per sec  4687 non-null   float64       
 5   Miss Dist.(Astronomical)      4687 non-null   float64       
 6   Orbit ID                      4687 non-null   int64         
 7   Orbit Determination Date      4687 non-null   datetime64[ns]
 8   Orbit Uncertainity            4687 non-null   int64         
 9   Minimum Orbit Intersection    4687 non-null   float64       
 10  Jupiter Tisserand Invariant   4687 non-null   float64       
 11  Epoch Osculation              

In [ ]:
scaler = StandardScaler()

In [ ]:
X = df.drop("Orbit Determination Date",axis=1)

In [ ]:
scaler.fit(X)

StandardScaler()

In [ ]:
X = scaler.transform(X)

In [ ]:
columns = list(df.columns)

In [ ]:
columns.remove("Orbit Determination Date")

In [ ]:
columns

['Absolute Magnitude',
 'Est Dia in KM(min)',
 'Est Dia in KM(max)',
 'Epoch Date Close Approach',
 'Relative Velocity km per sec',
 'Miss Dist.(Astronomical)',
 'Orbit ID',
 'Orbit Uncertainity',
 'Minimum Orbit Intersection',
 'Jupiter Tisserand Invariant',
 'Epoch Osculation',
 'Eccentricity',
 'Semi Major Axis',
 'Inclination',
 'Asc Node Longitude',
 'Orbital Period',
 'Perihelion Distance',
 'Perihelion Arg',
 'Aphelion Dist',
 'Perihelion Time',
 'Mean Anomaly',
 'Mean Motion']

In [ ]:
X = pd.DataFrame(X,columns=columns)

In [ ]:
X.head()

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Epoch Date Close Approach,Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit ID,Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
0,-0.231042,-0.209411,-0.209411,-1.973087,-1.077139,1.116075,-0.295089,0.481822,-0.631720,-0.341049,...,0.012875,-0.671955,1.377191,-0.070049,-0.021172,-1.223886,0.019571,0.459190,0.778393,-0.431100
1,-0.334824,-0.158406,-0.158406,-1.973087,0.568146,0.865921,-0.190639,-0.167955,1.158650,0.323902,...,-0.558078,1.375315,-0.343192,-0.565393,-0.393267,1.247897,-0.514802,0.070818,-0.069093,0.312582
2,-0.680766,0.072789,0.072789,-1.970035,-0.874892,-1.411843,-0.164526,-1.142621,-0.434844,-0.403261,...,0.111735,-0.835467,0.845573,0.021563,0.567722,0.623011,-0.021323,0.415579,1.039404,-0.522114
3,1.775418,-0.529864,-0.529864,-1.966982,-0.383539,0.195800,-0.556216,0.806711,-0.850681,0.029805,...,-0.275446,-0.500038,-1.113477,-0.327568,0.704524,-1.596341,-0.482690,0.184540,-1.045928,-0.110818
4,-0.231042,-0.209411,-0.209411,-1.966982,-0.566337,1.036159,-0.086188,-0.817732,-0.526325,0.079090,...,-0.333237,0.312713,-0.847599,-0.377402,0.637529,-0.248001,-0.529316,0.091457,-0.428186,-0.034580


In [ ]:
X.shape

(4687, 22)

# 3.Models


## 3.1 Neural Network Model


In [ ]:
class AstroidModel0(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(in_features=22,out_features=44)
    #self.layer2 = nn.Linear(in_features=44,out_features=44)
    #self.layer3 = nn.Linear(in_features=44,out_features=22)
    self.layer4 = nn.Linear(in_features=44,out_features=1)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(p=0.5)

  def forward(self,X):
    X = self.layer1(X)
    X = self.relu(X)
    #X = self.layer2(X)
    #X = self.relu(X)
    #X = self.layer3(X)
    #X = self.drop(X)
    X = self.layer4(X)

    return X

In [ ]:
model_0 = AstroidModel0()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),lr=0.001)

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
Y = Y.replace({True: 1, False: 0})

In [ ]:
X_tensor = torch.tensor(X.values,dtype=torch.float)
Y_tensor = torch.tensor(Y.values,dtype=torch.float)

In [ ]:
Y_tensor = Y_tensor.unsqueeze(dim=1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_tensor,Y_tensor,test_size=0.2)

### 3.1.1 Train & Test

In [ ]:
epochs = 441

for epoch in range(epochs):
  model_0.train()

  train_logits = model_0(X_train)

  train_loss = loss_fn(train_logits,Y_train)

  train_pred = torch.round(torch.sigmoid(train_logits))

  train_acc = accuracy_fn(train_pred,Y_train)

  optimizer.zero_grad()

  train_loss.backward()

  optimizer.step()

  if epoch%20 ==0:
    model_0.eval()
    with torch.inference_mode():

      test_logits = model_0(X_test)
      test_loss = loss_fn(test_logits,Y_test)

      test_pred = torch.round(torch.sigmoid(test_logits))

      test_acc = accuracy_fn(test_pred,Y_test)

      print(f"epoch:{epoch} loss_train:{train_loss} acc_train:{train_acc} loss_test:{test_loss} acc_test{test_acc}")


epoch:0 loss_train:0.672426700592041 acc_train:57.50866897839424 loss_test:0.6683121919631958 acc_test60.447761194029844
epoch:20 loss_train:0.5384044051170349 acc_train:85.142704721259 loss_test:0.542441725730896 acc_test83.7953091684435
epoch:40 loss_train:0.44506600499153137 acc_train:84.76927180581488 loss_test:0.45352962613105774 acc_test83.7953091684435
epoch:60 loss_train:0.38469254970550537 acc_train:84.76927180581488 loss_test:0.395842045545578 acc_test84.32835820895522
epoch:80 loss_train:0.347009539604187 acc_train:85.2760736196319 loss_test:0.3603020906448364 acc_test84.43496801705757
epoch:100 loss_train:0.32086244225502014 acc_train:85.78287543344892 loss_test:0.3362312316894531 acc_test84.43496801705757
epoch:120 loss_train:0.30002492666244507 acc_train:86.79647906108295 loss_test:0.31716328859329224 acc_test84.9680170575693
epoch:140 loss_train:0.2817727327346802 acc_train:87.5433448919712 loss_test:0.30050137639045715 acc_test86.24733475479745
epoch:160 loss_train:0.26

In [ ]:
model_0.eval()
with torch.inference_mode():

  logits = model_0(X_test)

  pred = torch.round(torch.sigmoid(logits))

In [ ]:
print(confusion_matrix(Y_test,pred))
print(classification_report(Y_test,pred))

[[765  17]
 [ 25 131]]
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.97       782
         1.0       0.89      0.84      0.86       156

    accuracy                           0.96       938
   macro avg       0.93      0.91      0.92       938
weighted avg       0.95      0.96      0.95       938



## 3.2 KNN Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNN = KNeighborsClassifier(3)

In [ ]:
X_tr,X_ts,Y_tr,Y_ts = train_test_split(X,Y,test_size=0.2)

### 3.2.1 Train & Test

In [ ]:
KNN.fit(X_tr,Y_tr)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn_pred = KNN.predict(X_ts)

In [ ]:
print(confusion_matrix(Y_ts,knn_pred))
print(classification_report(Y_ts,knn_pred))

[[743  38]
 [ 69  88]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       781
           1       0.70      0.56      0.62       157

    accuracy                           0.89       938
   macro avg       0.81      0.76      0.78       938
weighted avg       0.88      0.89      0.88       938

